# Histogram results analysis

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

def plt_clean_legend():
    original = plt.gca().get_legend_handles_labels()
    result = ([], [])
    for artist, label in zip(*original):
        try:
            if label == result[1][-1]:
                continue
        except:
            pass

        result[0].append(artist)
        result[1].append(label)
    plt.legend(result[0], result[1], loc="lower left")


In [ ]:
df = pd.read_csv("./histogram.csv") #, index_col=[0,1,2])
df

# Article image generation

In [ ]:
sns.set(style="whitegrid")

# First and last should be consistent, and reverse for consistency
palette = sns.color_palette(n_colors=5)
palette = [palette[0], palette[2], palette[4]]
palette = list(reversed(palette))

sns.barplot(x='mode', y='histogram',
            data=df[(df.numpoints == 2000000000)],
            capsize=0.1,
            order=reversed(['mm', 'ad', 'ad-ram']),
            palette=palette)

plt.ylabel("execution time (seconds)")
plt.xlabel(None)
plt.xticks(rotation=45)
plt.savefig("histogram-small.pdf", bbox_inches='tight')
plt.show()

sns.barplot(x='mode', y='histogram',
            data=df[(df.numpoints == 8000000000)],
            capsize=0.1,
            order=reversed(['mm', 'ad']),
            palette=palette[1:])

plt.ylabel("execution time (seconds)")
plt.xlabel(None)
plt.xticks(rotation=45)
plt.savefig("histogram-big.pdf", bbox_inches='tight')
plt.show()


In [ ]:
df[(df.numpoints == 2000000000)].groupby("mode").mean()

In [ ]:
df[(df.numpoints == 8000000000)].groupby("mode").mean()